In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

# Importing AnimalShelter class from AnimalShelter file
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser" # Hardcoded username
password = "password" # Hardcoded password
shelter = AnimalShelter(username, password) # Instantiating shelter object

# Uses read method from AnimalShelter class to read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable and Geolocation Chart')

# Adding Grazioso Salvare Logo to HTML dashboard
image_filename = 'Grazioso Salvare Logo.png' # Image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height': '25%', 'width': '25%'}),style={'textAlign': 'center'}),
    html.Div(html.Center(html.B(html.H1('Spencer Runde SNHU CS-340 Dashboard with Pie Chart and Map')))),
    
    html.Hr(),
    html.Div(dcc.RadioItems(
                id='filter-type',
                options = [
                    {'label': 'Water Rescue', 'value': '1'},
                    {'label': 'Mountain Rescue', 'value': '2'},
                    {'label': 'Disaster Rescue', 'value': '3'},
                    {'label': 'Reset', 'value': '4'},
                    ],
                value = '', # Setting default value to none
                labelStyle={'display': 'inline-block'} # Displaying filtering options in a row
    )),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Setting up features for the interactive data table to make it user-friendly for client
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 5,    
    ),
    
    html.Br(),
    html.Hr(),
    
# Display pie chart and geolocation chart side by side in a row
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Datatable filter callback
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):

    if filter_type == '1':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))

    elif filter_type == '2':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))

    elif filter_type == '3':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))

    else:
        df = pd.DataFrame.from_records(shelter.read({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)

# Selected columns callback
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Breed pie chart callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    # Adding code for pie chart
    dff = px.data
    return [
        dcc.Graph(
            figure = px.pie(data_frame=viewData, names='breed')
        )
    ]

# Geolocation chart callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
# Adding code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Marking Austin as center, because most locational data is centered around that point
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'], dff.loc[0,'location_long']], children=[ # Pulls lat and long from first row for marker position
                dl.Tooltip(dff.iloc[0,4]), # Pulls animal breed from first row
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9]) # Pulls animal name from first row
                ])
            ])
        ])
    ]

app


ModuleNotFoundError: No module named 'jupyter_plotly_dash'